# Minicurso: O que o twitter está pensando?

Estes tutoriais apresentam os principais scritps desenvolvidos no minicurso: **O que o twitter está pensando? Extraindo informações em redes sociais utilizando Python**. Os arquivos completos dos scritps e códigos gerados podem ser encontrados nas pastas **scritps** e **web** na raiz do repositório.

A apresentação referente a este minicurso está disponível no site: www.data2learning.com/cursos.

## 02 - Coletando Dados do Twitter (Parte 2)

Na primeira parte do tutorial vimos como utilizar o método **search** para realizar a coleta de dados. Esse método permite buscar os últimos *n* twitters publicados, sendo esse *n*, normalmente, igual a 100. No entanto, para tarefas que estamos querendo fazer seria interessante deixar a coleta rodando e toda vez que algo do nosso interesse fosse postado, fosse coletado pelo nosso aplicativo. Para fazer isso, o twitter tem a chamada ***Streaming API***. Desta forma, a API do twitter permite aos desenvolvedores uma ferramenta em tempo real para coleta de dados. 

Informações sobre o streaming podem ser encontradas em: https://dev.twitter.com/streaming/overview

Vamos criar um novo arquivo python: **streamingtwitter.py**.

In [9]:
# O código a seguir não precisa ser inserido, ele é utilizado somente para que alguns imports funcionem na estrutura do Jupyter
import sys
sys.path.append('/Users/adolfoguimaraes/Desenvolvimento/d2l/minicursotwitter/minicursotwitter/scripts')

In [2]:
# O primeiro passo é importar o Streammer e a API do Twitter
from twython import TwythonStreamer
from twython import Twython

A classe *MyStreammer* extende a classe *TwythonStreamer* do twython. Essa classe tem basicamente dois métodos: **onsuccess** e **onerror**. No primeiro iremos definir as ações para quando a coleta for realizada com sucesso. O segundo serve para fazer o tratamento de error em caso de falha na busca. 

O nosso arquivo ficaria como descrito a seguir.

In [6]:
#coding=utf8
from twython import TwythonStreamer
from twython import Twython

class MyStreamer(TwythonStreamer):

    def on_success(self, data):

        print(data)

    def on_error(self, status_code, data):

        print(status_code)

        self.disconnect()


No método **on_success** apenas imprimimos o resultado da busca. A variável *data* retorna uma estrutura igual a que foi obtida quando utilizamos o método **search** na primeira etapa. Para testar a classe, vamos criar mais um arquivo python para que possamos chamar esta classe. Criem o arquivo *collecttwitter_stream.py* e inclua o código a seguir.

In [10]:
#Primeiro precisamos importar o Stramer criado no arquivp "streamingtwitter.py"
from streamingtwitter import MyStreamer

#Definição das chaves da API do Twitter
APP_KEY = None # Get Keys and Access Token at apps.twitter.com
APP_SECRET = None # Get Keys and Access Token at apps.twitter.com
OAUTH_TOKEN = None # Get Keys and Access Token at apps.twitter.com
OAUTH_TOKEN_SECRET = None # Get Keys and Access Token at apps.twitter.com

#E criamos nossa instância do coletor
stream = MyStreamer(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

Para iniciar a busca vamos utilizar a chamada **statuses.filter** do objeto *stream* criado. Como parâmetro vamos fazer os termos que desejamos pesquisar. Quando executado, a API vai criar tipo um "listener". A registro de um twitter com os termos especificados, o método **on_sucess** da sua classe *MyStreamer* vai ser chamado. O código a seguir vai coletar todos os tweets com a palavra *brasil*.

In [ ]:
stream.statuses.filter(track="brasil")

Podemos colocar também uma lista de termos. Nesse caso, todos os tweets que tenham pelo menos um destes termos serão coletados.

In [ ]:
stream.statuses.filter(track="brasil, #brasil, aracaju, #aracaju")

Observe que a execução do **statuses.filter** incializa um processo que ficará em execução até que seja interrompido pelo usuário. Se essa não for a intensão do usuário, ele deve utilizar o método **search** como descrito anteriormente.

A API do twitter permite coletar da mesma forma o que um usuário ou uma lista de usuários publica. O método é o mesmo, no entanto devemos inserir o parâmetro **follow** na busca.

In [ ]:
# Pode ser passado um usuário
stream.statuses.filter(follow="20678688")

# Ou uma lista de usuários separados por vírgula
stream.statuses.filter(follow="20678688,736392442384154624")

Para descobrir o id do usuário você pode utilizar a própria API através do método **show_user**. Alguns sites permitem pegar esta informação, como por exemplo, http://mytwitterid.com/.

Um vez que já sabemos como coletar os dados, precisamos melhorar a apresentação dos dados coletados. Os métodos de busca trazem muitas informações, mas nem sempre queremos exbir todas. Vale ressaltar que isso depende de cada aplicação. Sendo assim, a melhor maneneira de trabalhar com estes dados é visitar sempre a API do twitter para verificar as informações que são coletadas e a partir daí analisar quais as informações são interessantes. Neste momento, vamos trabalhar somente com as informações que são relevantes para a proposta deste curso. 

Voltemos ao método **on_success** da classe *MyStreamer*. 

Este método é executado toda vez que um twitter com os critérios de busca especificado é postado. No nosso exemplo, apenas imprimimos o que é coletado no console. Vamos modificar este método para que ele imprima as informações que são de interesse e de forma mais organizada. 

Nosso arquivo **streamingtwitter.py** ficaria assim:

In [11]:
# coding=utf8

# Import dos pacotes necessários do Twython
from twython import TwythonStreamer
from twython import Twython

# Import métodos para tratar a data
from datetime import datetime
import pytz


class MyStreamer(TwythonStreamer):
    def on_success(self, data):
        # Id do twitter
        object_id = data['id']

        # Id do usuário que postou o texto
        user_id = data['user']['id']

        # Usuário que postou o texto
        user_name = data['user']['screen_name']

        # Texto postado em utf-8
        text = data['text'].encode('utf-8')

        # Data que foi publicado
        posted_at_tweet = data['created_at']

        # Data que foi publicado formatada
        fmt = '%Y-%m-%d %H:%M:%S.%f'
        new_date = datetime.strptime(posted_at_tweet, '%a %b %d %H:%M:%S +0000 %Y').replace(tzinfo=pytz.UTC)

        published_date = str(new_date.strftime(fmt))

        tweet = {
            'object_id': object_id,
            'user_id': user_id,
            'user_name': user_name,
            'text': text,
            'published_date': published_date,
            'create_date': published_date,
            'last_update': published_date,
        }

        print(tweet)

    def on_error(self, status_code, data):
        print(status_code)

        self.disconnect()

Com o que vimos até aqui é possível fazer uma coleta de dados do twitter sobre qualquer assunto. Claro que a API permite outras funcionalidades que fogem um pouco do escopo deste curso. Para os interessados, explorem mais a API em relação aos compos que ela retorna em cada busca e veja a documentação para mais detalhes.

Uma vez que já sabemos como coletar, o nosso próximo passo é processar esse texto. Para esta etapa utilizaremos a API NLTK (Natural Language ToolKit) disponível em: http://www.nltk.org/.